In [ ]:
import numpy as np
import pandas as pd 

import data_api
import helpers

## 1. Loading the datasets

In [ ]:
# Create the data_api client and set the number of records to be returned
client = data_api.Client()
num_records = 100

# Note: since we are not providing an api token to the client, it is best to keep the number 
# of records returned low

## 311 Service Requests from 2010 to Present

In [ ]:
df_311 = client.load_311(
    fetch_all=True, 
    select="created_date, closed_date, complaint_type, descriptor, x_coordinate_state_plane,    \
                status, due_date, bbl, y_coordinate_state_plane, latitude, longitude", 
    where=
        "agency IN ('DOB', 'HPD') AND incident_zip IN ('11212', '11233')"
)
df_311

In [ ]:
df_complaint_problems = client.load_complaint_problems(
    fetch_all=True, 
    where="unittypeid=92"
)
df_complaint_problems.head()

In [ ]:
# NOTE: the DOB Complaints Received dataset zip_code column has extra spaces appended to the string, so we are using the starts_with function from the SoQL API to filter the results.

# starts_with documentation: https://dev.socrata.com/docs/functions/starts_with.html

df_dob_complaints = client.load_dob_complaints(
    fetch_all=True,
    select=
        "complaint_number, date_entered, house_number, house_street,            \
            bin, complaint_category, unit, disposition_date, disposition_code,  \
            inspection_date, dobrundate",
    where=
        "starts_with(zip_code, '11212') OR                                      \
            starts_with(zip_code, '11233')"
) 
df_dob_complaints.head()

In [ ]:
print(min(df_dob_complaints["date_entered"]), max(df_dob_complaints["date_entered"]))

In [ ]:
df_311 = client.load_311(fetch_all=True, select=["Created Date"])
# df_complaint_problems = client.load_complaint_problems(limit=num_records)
# df_dob_omplaints = client.load_dob_complaints(limit=num_records)

In [ ]:
df_311.head()

In [ ]:
df_complaint_problems.head()

In [ ]:
df_dob_omplaints.head()

## 2. Extracting useful columns